In [1]:
import os  # Importing the os module for file and directory operations
import requests  # Importing the requests module for making HTTP requests
import json  # Importing the json module for working with JSON data
import openai  # Importing the openai module for accessing the OpenAI API
import datetime  # Importing the datetime module for working with dates and times
import itertools  # Importing the itertools module for generating combinations


In [30]:
# Define API key
file_path = "C:\\Users\\otavi\\Documents\\My API key open AI.txt"

# Open the file and read its content
with open(file_path, 'r') as file:
    api_key = file.readline().strip()

print("API key loaded successfully!")


# The base URL for the OpenAI API
api_url = "https://api.openai.com/v1/chat/completions"

def call_api(prompt):
    """
    Function to call OpenAI API and return the response
    """
    openai.api_key = api_key  # Setting the OpenAI API key
    data = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    }
    response = openai.ChatCompletion.create(**data)  # Calling the OpenAI API
    return response['choices'][0]['message']['content'].strip()  # Returning the response content


API key loaded successfully!


In [31]:
# The number of API calls for each prompt
num_calls = 5  # Change this to the number you want

In [32]:
# Define a dictionary of transformation functions
# Define a dictionary of transformation functions
TRANSFORMATIONS = {
    'bold': ("bold", lambda x: f"**{x}**"),  # Function to transform text to bold format
    'italic': ("italic", lambda x: f"_{x}_"),  # Function to transform text to italic format
    "long_dash": ("long_dash", lambda x: f'— {x} —'),  # Function to add long dashes around text
    "asterisks": ("asterisks", lambda x: f'* {x} *'),  # Function to add asterisks around text
    "quotes": ("quotes", lambda x: f'"{x}"'),  # Function to add quotes around text
    "tilde": ("tilde", lambda x: f'~{x}~'),  # Function to add tildes around text
    "paranthesis": ("paranthesis", lambda x: f'({x})'),  # Function to add parantheses around text
    "codeblock": ("codeblock", lambda x: f'`{x}`'),  # Function to format text as a code block
    "focus": ("focus", lambda x: f'>>> {x} <<<'),  # Function to add focus around text
    'capslock': ("capslock", lambda x: x.upper()),  # Convert the text to uppercase
    'spacing': ("spacing", lambda x: ' '.join(x)),  # Add space between each character of the text
}

In [40]:

import os
import datetime
import re

def create_prompt_folder(word_to_transformations,transformations):
    def remove_special_characters(input_string):
        return re.sub(r'\W+', '_', input_string)
    # Define the base directory where folders will be saved
    base_directory = "C:\\Users\\otavi\\Documents\\October Research Prompts"
    
    # Create a timestamp for uniqueness
    timestamp = datetime.datetime.now().strftime('%m%d_%H%M')
            # Sanitize word and transformations
    words_str = "_".join([remove_special_characters(word) for word in word_to_transformations.keys()])
    sanitized_transformations = [remove_special_characters(t) for t in transformations]
    # If no transformations, name the folder "no_transformations"
    if not transformations:
        folder_name = f"{timestamp}_{words_str}_no_transformations"
    else:


        # Get the first 3 characters of each transformation
        transformations_str = "_".join([t[:3] for t in sanitized_transformations])

        # Construct the folder name using timestamp, word, and transformations
        folder_name = f"{timestamp}_{words_str}_{transformations_str}"

    
    # Combine the base directory and the folder name
    full_path = os.path.join(base_directory, folder_name)
    
    # Create the directory
    try:
        os.makedirs(full_path)
    except Exception as e:
        print(f"Could not create the directory: {full_path}. Error: {e}")
        full_path = None

    return full_path



In [34]:
def write_prompt_to_file(prompt, folder_name, transformations):
    """
    Function to write the prompt to a text file
    """
    transformations_str = ", ".join([TRANSFORMATIONS[t][0] for t in transformations])
    with open(os.path.join(folder_name, "PROMPT.txt"), "w") as file:
        file.write(f"Prompt: {prompt}\nTransformations: {transformations_str}")


In [43]:
def transform_prompt(base_prompt, word_to_transformations):
    """
    Function to apply transformations to the base_prompt
    """
    transformed_prompt = base_prompt

    content_transformations = ['capslock', 'spacing']

    for word, transformations in word_to_transformations.items():
        current_word = word  # Initialize with the original word
        
        # Apply content changing transformations first
        for transformation in content_transformations:
            if transformation in transformations:
                # Apply the transformation to the current_word and update it
                new_word = TRANSFORMATIONS[transformation][1](current_word)
                # Update the transformed_prompt with the new current_word state
                transformed_prompt = transformed_prompt.replace(current_word, new_word)
                current_word = new_word  # Update current_word to the latest transformed state
        
        # Apply non-content changing transformations next
        for transformation in transformations:
            if transformation not in content_transformations:
                new_word = TRANSFORMATIONS[transformation][1](current_word)
                transformed_prompt = transformed_prompt.replace(current_word, new_word)
                current_word = new_word  # Update current_word to the latest transformed state

    return transformed_prompt

In [36]:
def get_all_combinations(transformations):
    """
    Function to get all combinations of transformations
    """
    all_combinations = []  # create an empty list to store all combinations
    for i in range(1, len(transformations)+1):  # iterate over the range from 1 to the length of transformations + 1
        for subset in itertools.combinations(transformations, i):  # iterate over all combinations of transformations with length i
            all_combinations.append(subset)  # append the current combination to the all_combinations list
    return all_combinations  # return the list of all combinations

import itertools

def ranked_combinations(elements):
    all_combinations = []
    
    for r in range(1, len(elements) + 1):
        combinations = itertools.combinations(elements, r)
        all_combinations += [list(combination) for combination in combinations]
                
    return all_combinations


In [37]:
def call_api_and_store_responses(base_prompt, word_to_transformations, all_combinations):
    for transformation_list in all_combinations:
        transformed_word_to_transformations = {}
        for word in word_to_transformations:
            transformed_word_to_transformations[word] = [t for t in word_to_transformations[word] if t in transformation_list]

        # Generate the transformed prompt
        prompt = transform_prompt(base_prompt, transformed_word_to_transformations)

        # Create a unique folder for the transformed prompt
        folder_name = create_prompt_folder(word_to_transformations,transformation_list)
        
        # Write the prompt to a text file
        write_prompt_to_file(prompt, folder_name, transformation_list)
        
        # Call the API and write the responses to text files
        for j in range(num_calls):
            response = call_api(prompt)
            with open(os.path.join(folder_name, f"{j+1}.txt"), "w") as file:
                file.write(response)


In [49]:
base_prompts = [
    "Write a two paragraph story based on the following sentence: he was a smart and handsome man"
]

word_to_transformations = {
    #"smart":[]
    'smart': ['spacing',"capslock","bold"]  # Mapping the word 'smart' to a list of transformations
    #'handsome': ['bold', 'italic', 'codeblock', 'focus', 'quotes', 'tilde', 'paranthesis', 'asterisks', 'long_dash'],  # Mapping the word 'handsome' to a list of transformations
}

for base_prompt in base_prompts:
    transformations_for_combinations = list(set(sum(word_to_transformations.values(), [])))
    
    # Add an empty list to handle the case where there are no transformations
    all_combinations = [ [] ] + ranked_combinations(transformations_for_combinations)
    call_api_and_store_responses(base_prompt, word_to_transformations, all_combinations)
